In [1]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import confusion_matrix, classification_report

In [3]:
data = pd.read_csv("SCMS_Delivery_History_Dataset.csv")
data

,ID,Project Code,PQ #,PO / SO #,ASN/DN #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,PO Sent to Vendor Date,Scheduled Delivery Date,Delivered to Client Date,Delivery Recorded Date,Product Group,Sub Classification,Vendor,Item Description,Molecule/Test Type,Brand,Dosage,Dosage Form,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,2-Jun-06,2-Jun-06,2-Jun-06,HRDT,HIV test,RANBAXY Fine Chemicals LTD.,"HIV, Reveal G3 Rapid HIV-1 Antibody Test, 30 T...","HIV, Reveal G3 Rapid HIV-1 Antibody Test",Reveal,NaN,Test kit,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,14-Nov-06,14-Nov-06,14-Nov-06,ARV,Pediatric,Aurobindo Pharma Limited,"Nevirapine 10mg/ml, oral suspension, Bottle, 2...",Nevirapine,Generic,10mg/ml,Oral suspension,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,Pre-PQ Process,Date Not Captured,27-Aug-06,27-Aug-06,27-Aug-06,HRDT,HIV test,Abbott GmbH & Co. KG,"HIV 1/2, Determine Complete HIV Kit, 100 Tests","HIV 1/2, Determine Complete HIV Kit",Determine,NaN,Test kit,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,1-Sep-06,1-Sep-06,1-Sep-06,ARV,Adult,SUN PHARMACEUTICAL INDUSTRIES LTD (RANBAXY LAB...,"Lamivudine 150mg, tablets, 60 Tabs",Lamivudine,Generic,150mg,Tablet,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,Date Not Captured,11-Aug-06,11-Aug-06,11-Aug-06,ARV,Adult,Aurobindo Pharma Limited,"Stavudine 30mg, capsules, 60 Caps",Stavudine,Generic,30mg,Capsule,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10319,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,10/16/14,N/A - From RDC,31-Jul-15,15-Jul-15,20-Jul-15,ARV,Pediatric,SCMS from RDC,"Lamivudine/Nevirapine/Zidovudine 30/50/60mg, d...",Lamivudine/Nevirapine/Zidovudine,Generic,30/50/60mg,Chewable/dispersible tablet - FDC,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",No,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
10320,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,10/24/14,N/A - From RDC,31-Jul-15,6-Aug-15,7-Aug-15,ARV,Adult,SCMS from RDC,"Lamivudine/Zidovudine 150/300mg, tablets, 60 Tabs",Lamivudine/Zidovudine,Generic,150/300mg,Tablet - FDC,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,No,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
10321,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,8/12/14,N/A - From RDC,31-Aug-15,25-Aug-15,3-Sep-15,ARV,Adult,SCMS from RDC,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...,Generic,600/300/300mg,Tablet - FDC,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,No,Weight Captured Separately,Freight Included in Commodity Cost,5284.04
10322,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,7/1/15,N/A - From RDC,9-Sep-15,4-Aug-15,11-Aug-15,ARV,Adult,SCMS from RDC,"Lamivudine/Zidovudine 150/300mg, tablets, 60 Tabs",Lamivudine/Zidovudine,Generic,150/300mg,Tablet - FDC,60,17465,1138

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10324 entries, 0 to 10323
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            10324 non-null  int64  
 1   Project Code                  10324 non-null  object 
 2   PQ #                          10324 non-null  object 
 3   PO / SO #                     10324 non-null  object 
 4   ASN/DN #                      10324 non-null  object 
 5   Country                       10324 non-null  object 
 6   Managed By                    10324 non-null  object 
 7   Fulfill Via                   10324 non-null  object 
 8   Vendor INCO Term              10324 non-null  object 
 9   Shipment Mode                 9964 non-null   object 
 10  PQ First Sent to Client Date  10324 non-null  object 
 11  PO Sent to Vendor Date        10324 non-null  object 
 12  Scheduled Delivery Date       10324 non-null  object 
 13  D

# Preprocessing

In [127]:
def preprocess_input(df):
    df = df.copy()
    
    #Dropping columns
    df = df.drop('ID', axis=1)
    df = df.drop(['PQ First Sent to Client Date','PO Sent to Vendor Date'], axis=1)
    df = df.drop(['Weight (Kilograms)', 'Freight Cost (USD)'], axis=1)
    df = df.drop(['PQ #','PO / SO #','ASN/DN #'], axis=1)
    
    #Missing rows in target column
    missing_rows_target = df[df['Shipment Mode'].isna()].index
    df = df.drop(missing_rows_target, axis=0).reset_index(drop=True)
    
    #Filling Dosage columnn with its most recent mode
    df['Dosage'] = df['Dosage'].fillna(df['Dosage'].mode()[0])
    df['Line Item Insurance (USD)'] = df['Line Item Insurance (USD)'].fillna(df['Line Item Insurance (USD)'].mean())
    
    #Extracing Datetime from date columns
    for column in ['Scheduled Delivery Date', 'Delivered to Client Date', 'Delivery Recorded Date']:
        df[column] = pd.to_datetime(df[column])
        df[column + ': ' + 'Year'] = df[column].apply(lambda x: x.year)
        df[column + ': ' + 'Month'] = df[column].apply(lambda x: x.month)
        df[column + ': ' + 'Day'] = df[column].apply(lambda x: x.day)
        df = df.drop(column, axis=1)
        
    #Binary Encoding
    
    df['Fulfill Via'] = df['Fulfill Via'].replace({'Direct Drop': 0, 'From RDC': 1})
    df['First Line Designation'] = df['First Line Designation'].replace({'No': 0, 'Yes': 1})
        
    #one hot encoding   
    for column in df.select_dtypes('object').columns.drop('Shipment Mode'):
        dummies = pd.get_dummies(df[column], prefix=column)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    
    #Splitting the data
    y = df['Shipment Mode']
    X = df.drop('Shipment Mode', axis=1)
    
    #Scaling X using StandardScaler
    scale = StandardScaler()
    X = pd.DataFrame(scale.fit_transform(X), columns=X.columns)
    
    #Training
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)
    

    return X_train, X_test, y_train, y_test


In [128]:
X_train, X_test, y_train, y_test = preprocess_input(data)
X_train

Fulfill Via  Unit of Measure (Per Pack)  Line Item Quantity  \
8905     0.978159                   -0.228115           -0.423801   
1414     0.978159                   -0.228115            0.195635   
1409     0.978159                   -0.228115            0.527154   
874     -1.022329                    0.295513           -0.447364   
1941     0.978159                   -0.620836            0.157430   
...           ...                         ...                 ...   
2895    -1.022329                   -0.228115           -0.405536   
7813     0.978159                   -0.228115           -0.006481   
905     -1.022329                   -0.620836            1.596889   
5192    -1.022329                    0.295513           -0.280249   
235     -1.022329                   -0.228115           -0.452368   

      Line Item Value  Pack Price  Unit Price  First Line Designation  \
8905        -0.437730   -0.377054   -0.222578                0.681493   
1414         0.410434   -0.226095   -0.174597                0.681493   
1409         0.359278   -0.319733   -0.205130                0.681493   
874         -0.271777    2.565557    0.348832                0.681493   
1941         0.331224   -0.234846   -0.096083                0.681493   
...               ...         ...         ...                     ...   
2895        -0.431585   -0.389087   -0.226940               -1.467367   
7813        -0.327837   -0.422123   -0.240026                0.681493   
905          2.210789   -0.231127   -0.096083                0.681493   
5192         1.389344    1.471650    0.130736                0.681493   
235         -0.444458   -0.135957   -0.144064               -1.467367   

      Line Item Insurance (USD)  Scheduled Delivery Date: Year  \
8905                  -0.470402                       1.081324   
1414                   0.705524                      -0.999784   
1409                   0.635330                      -0.999784   
874                   -0.277157                      -1.416006   
1941                   0.596825                      -0.999784   
...                         ...                            ...   
2895                  -0.465973                       1.081324   
7813                  -0.357735                       0.248881   
905                    2.504060                      -1.416006   
5192                   2.302216                      -0.583563   
235                   -0.467696                      -0.999784   

      Scheduled Delivery Date: Month  Scheduled Delivery Date: Day  \
8905                        0.762230                      1.300091   
1414                        0.458083                     -1.464813   
1409                       -0.150212                     -0.188703   
874                        -1.670947                      0.023981   
1941                       -0.454359                     -1.039443   
...                              ...                           ...   
2895                        1.066377                      0.236666   
7813                        0.762230                      1.087406   
905                         0.762230                      0.555694   
5192                        1.370524                     -0.614073   
235                        -1.062653                      1.300091   

      Delivered to Client Date: Year  Delivered to Client Date: Month  \
8905                        1.081853                         0.157115   
1414                       -0.991462                         0.457709   
1409                       -0.991462                        -0.143479   
874                        -1.406125                        -1.646450   
1941                       -0.991462                        -0.444073   
...                              ...                              ...   
2895                        1.081853                         1.058898   
7813                        0.252527                         1.058898   
905      

In [129]:
models = {LogisticRegression(): 'Logistic Regression', SVC(): 'Support Vector Machine', MLPClassifier(): 'Neural Network'}
for model in models.keys():
    model.fit(X_train, y_train)

/Users/gautamjaladi/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/gautamjaladi/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [130]:
for model, name in models.items():
    print(name + " " + "{:.3f}%".format(model.score(X_test, y_test)* 100))

Logistic Regression 88.428%
Support Vector Machine 87.057%
Neural Network 89.967%
